In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import pickle

# Separar las variables independientes (X) y la variable objetivo (y)
X = df2.drop(['price'], axis=1)
y = df2['price']

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de regresión polinomial y entrenarlo
grado_polinomial = 2  # Puedes ajustar el grado del polinomio según sea necesario
poly = PolynomialFeatures(degree=grado_polinomial)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Crear el modelo de árbol de decisión
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train_poly, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test_poly)

# Calcular el error cuadrático medio (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)

# Obtener los nombres de las columnas con las que se entrenó el modelo
columnas_entrenamiento = poly.get_feature_names_out(input_features=X.columns)  # Obtener los nombres de las columnas polinomiales

# Imprimir el RMSE
print("RMSE:", rmse)

# Guardar el modelo, el RMSE y el nombre de las columnas en un archivo pickle
with open('modelo_arbol_decision.pkl', 'wb') as archivo:
    pickle.dump((model, rmse, columnas_entrenamiento), archivo)

In [ ]:
import pickle

# Guardar el modelo, el RMSE y el nombre de las columnas en un archivo pickle
with open('modelo_entrenado.pkl', 'wb') as archivo:
    pickle.dump((model, rmse, columnas_entrenamiento), archivo)

In [ ]:
# Cargar el modelo, el RMSE y la lista de columnas desde el archivo pickle
with open('modelo_arbol_decision.pkl', 'rb') as archivo:
    model, rmse, columnas_entrenadas = pickle.load(archivo)

# Crear el objeto PolynomialFeatures con el grado polinomial utilizado para entrenar el modelo
grado_polinomial = 2  # Asegúrate de usar el mismo grado polinomial que se usó en el entrenamiento
poly = PolynomialFeatures(degree=grado_polinomial)

# Ajustar el objeto poly con los datos de entrenamiento
X_train_poly = poly.fit_transform(X_train)

def obtener_prediccion(generos, earlyaccess=True):
    # Crear un DataFrame con todas las columnas polinomiales generadas durante el entrenamiento
    datos_usuario = pd.DataFrame(columns=columnas_entrenadas)
    datos_usuario.loc[0] = 0

    # Asignar 1 a los géneros proporcionados por el usuario
    for genero in generos:
        if f'genre_{genero}' in datos_usuario.columns:
            datos_usuario[f'genre_{genero}'] = 1

    # Asignar el valor de early_access proporcionado por el usuario
    datos_usuario['early_access'] = earlyaccess

    # Aplicar la transformación polinomial a los datos de entrada
    X_usuario_poly = poly.transform(datos_usuario)
    X_usuario_poly = pd.DataFrame(X_usuario_poly, columns=poly.get_feature_names(columnas_entrenadas))

    # Realizar la predicción utilizando el modelo
    precio_prediccion = model.predict(X_usuario_poly)

    # Devolver el precio y el RMSE
    return {'precio': precio_prediccion.tolist(), 'rmse': rmse}

In [ ]:
# Ahora puedes llamar a la función obtener_prediccion con los datos
generos = ['Web Publishing']  # Ejemplo de géneros
resultado = obtener_prediccion(generos, earlyaccess=False)

print("Predicciones de precio:", resultado['precio'])
print("RMSE:", resultado['rmse'])